### GetBasisInfo函数解读
* 获取

In [ ]:
def GetBasisInfo(int_el:femDofel, ext_el_0:femDofel, ext_el_1:femDofel, Vint_el:femDofel, Vext_el_0:femDofel, Vext_el_1:femDofel, 
                    unvec, theta0, k, mu, nquad=4, rho=1, bub_opt=False,
                    jj = 0,theta_new=0, CoordsInt: np.array=None, CoordsExt: np.array=None):
    '''
        获取基函数的信息，包括基函数在积分节点上的值，梯度值，以及在边界上的跳跃值
        需要的是什么：
    '''
    # order and dimension of fes of U and p
    porder = int_el.order
    pspdim = int_el.spdim
    Uorder = Vint_el.order
    Uspdim = Vint_el.spdim
    
    pIntCoord = CoordsInt[int_el.vnr,:]
    pExtCoord0 = CoordsExt[ext_el_0.vnr,:]
    pExtCoord1 = CoordsExt[ext_el_1.vnr,:]
    # 激活的内部单元的顶点坐标，以及外部两个单元的坐标，下面三个是一样的
    UIntCoord = CoordsInt[Vint_el.vnr,:]
    UExtCoord0 = CoordsExt[Vext_el_0.vnr,:]
    UExtCoord1 = CoordsExt[Vext_el_1.vnr,:]

    pDof_els:List[femDofel] = [int_el, ext_el_0, ext_el_1]
    UDof_els:List[femDofel] = [Vint_el, Vext_el_0, Vext_el_1]

    # Get the coordinates of edges of the elements
    El_Co0 = pIntCoord[:2,:]
    El_Co1 = pExtCoord0[:2,:]
    El_Co2 = pExtCoord1[:2,:]
    r1, phi_in = GetRPhi(El_Co0, theta0, '+')
    r0, phi0 = GetRPhi(El_Co1, theta0, '-')
    r2, phi2 = GetRPhi(El_Co2, theta0, '-')
    assert(np.allclose(r1,r2))
    assert(np.allclose(r1,r0))

    # Proxy functions el objects
    pel_list:List[Element] = []
    pel_list.append( Element(int_el.nr,   pIntCoord,  '+', order=porder, dim=pspdim) )
    pel_list.append( Element(ext_el_0.nr, pExtCoord0, '-', order=porder, dim=pspdim) )
    pel_list.append( Element(ext_el_1.nr, pExtCoord1, '-', order=porder, dim=pspdim) )

    Uel_list:List[Element] = []
    Uel_list.append( Element(Vint_el.nr,   UIntCoord,  '+', order=Uorder, dim=Uspdim, bubble=bub_opt) )
    Uel_list.append( Element(Vext_el_0.nr, UExtCoord0, '-', order=Uorder, dim=Uspdim, bubble=bub_opt) )
    Uel_list.append( Element(Vext_el_1.nr, UExtCoord1, '-', order=Uorder, dim=Uspdim, bubble=bub_opt) )

    for ii in range(len(pel_list)):
        pel_list[ii].GetBasis()
    
    for ii in range(len(Uel_list)):
        Uel_list[ii].GetBasis()

    CurveEl_List:List[myCEl] = [ myCEl(r0, Modi(phi_in, phi0+theta0), [0,1]),
                                myCEl(r0, Modi(phi0+theta0, phi_in+theta0), [0,2])]
    
    G_X_list = []
    G_w_list = []
    for CEl in CurveEl_List:
        CEl.G_p, CEl.G_w = generate_Gauss_formula(CEl.theta[0],CEl.theta[1],nquad)
        CEl.G_X = np.array([[CEl.r*np.cos(thetai), CEl.r*np.sin(thetai)] for thetai in CEl.G_p])
        CEl.G_w = CEl.G_w*CEl.r
        G_X_list.append(CEl.G_X)
        G_w_list.append(CEl.G_w)

    # 检测求积节点和权重的改写正确性
    # Info_X, Info_W = QuadInfo(theta_new,theta0,nquad,1)
    # print("Confirm 1 {}".format(np.allclose(np.array(Info_X),np.array(G_X_list))))
    # print("Confirm 2 {}".format(np.allclose(np.array(Info_W),np.array(G_w_list))))
        
    # 以U_Proxy_Vals为例，作为一个list，两分量分别存储第一段和第二段曲线单元上的16个自由度基函数在求积节点上的值
    # 第一段曲线单元是在 phi_in, phi0+theta0 上，即 phi1, theta0/2 上
    # 第二段曲线单元是在 theta0/2 到 phi1 + theta0 上
    # 对第一段曲线单元情况，标量情况下的张量是 8x2 x quad 的，第一个指标对应 
    # 内部的线性基函数，bubble，外部的线性基函数，bubble再乘以向量值情形，第二个指标对应求积节点
    U_Proxy_Vals = []
    p_Proxy_Vals = []
    U_Proxy_grad_Vals = []
    
    for CEl in CurveEl_List:
        p_ProxyList:List[SProxy] = []
        U_ProxyList:List[VProxy] = []
        pdofsList = []
        UdofsList = []
        
        U_Proxy_Val = []
        p_Proxy_Val = []
        U_Proxy_grad_Val = []

        for ii in CEl.Els:
            pdofsList += pDof_els[ii].DofObj.GetDofs()
            UdofsList += UDof_els[ii].DofObj.GetDofs()
            p_ProxyList += pel_list[ii].GetProxy()
            U_ProxyList += Uel_list[ii].GetProxy()

        # 存储两个分量，仅仅表出scalar情形的 在每个自由度的基函数 在求积节点的值，是一个自由度x求积节点的矩阵
        # U_Dofs是先排完内部单元的向量基，然后再排列外部单元的向量基
        # 获取16个向量基函数在求积点的值，[基，积分点，向量值]
        for B_test in U_ProxyList:
            U_Proxy_Val.append(np.array(B_test.f.func(CEl.G_X)))
            # 每个基函数在各个求积节点上的矩阵
            [[a,b],[c,d]] = B_test.grad()
            U_Proxy_grad_Val.append(
                [   [[a.func(xx.reshape(1,-1))[0], b.func(xx.reshape(1,-1))[0]],
                     [c.func(xx.reshape(1,-1))[0], d.func(xx.reshape(1,-1))[0]]] for xx in CEl.G_X]
            )
    
        p_Proxy_Val = [
            list(p_test.f.func(CEl.G_X).reshape(1,-1)[0]) for p_test in p_ProxyList
        ]
        
        # res, coef1 = GetBasisVScale(pIntCoord,1,Info_X[kk])
        # res2, coef2 = GetBasisVScale(pExtCoord0,1,Info_X[kk])
        # # 把res和res2这两个矩阵转置然后竖着拼接，得到的矩阵是叠放的
        # p_new = np.vstack((res.T, res2.T))
        # print("Check p_proxy is {}".format(np.allclose(np.array(p_Proxy_Val),p_new)))
        # # 这里的p_new是6x nquad的矩阵，下面要基于它生成U_new，是个24x4x2的矩阵：规则如下：
        # # 1. 24个基函数，每个基函数在4个积分节点上的值，每个积分节点上的值是一个2维向量
        # # 2. 前12个是内部基函数，后12个是外部基函数
        # # 3. 前6个是x分量 （即y分量为0），后6个是y分量 
        # # 4. 前3个是线性基函数，后3个是edge基函数
        # U_new = np.zeros((24,nquad,2))
        # # 举例来说，最前面的三个是内部的x分量的线性基函数，因此这个3x4x2的矩阵 中的 3x4x1 正是p_new的前3行，另外3x4x1是0
        # U_new[:3,:,0] = p_new[:3,:]
        # # 接下来三个基函数是edge基函数，是p_new的前三行的两两乘积
        # # 例如第一个edge是0,2两个顶点所夹的边，是这两个基函数的乘积
        # # 第二个edge是2,1这两个顶点所夹的边，第三个edge是0，1这两个顶点所夹的边
        # U_new[3,:,0] = -1/2*p_new[0,:]*p_new[2,:]
        # U_new[4,:,0] = -1/2*p_new[1,:]*p_new[2,:]
        # U_new[5,:,0] = -1/2*p_new[0,:]*p_new[1,:]
        # # 接下来是y分量的基函数取值，是前面x分量的copy到y上
        # U_new[6:12,:,1] = U_new[:6,:,0]
        # # 接下来是外部的基函数取值，是采用的是p_new的后3行
        # U_new[12:15,:,0] = p_new[3:,:]
        # U_new[15,:,0] = -1/2*p_new[3,:]*p_new[5,:]
        # U_new[16,:,0] = -1/2*p_new[4,:]*p_new[5,:]
        # U_new[17,:,0] = -1/2*p_new[3,:]*p_new[4,:]
        # U_new[18:,:,1] = U_new[12:18,:,0]
        # print("Check U_procy is {}".format(np.allclose(U_new,np.array(U_Proxy_Val))))
        # # 下面改写U_grad的结果，这是一个24 x nquad x 2 x 2的矩阵，每个基函数在每个积分节点上的梯度
        # # grad( (phi1, phi2) ) = (grad phi1, grad phi2)
        # U_grad_new = np.zeros((24,nquad,2,2)) 
        # # 对前三个x分量基函数，y分量的梯度是0，线性函数 ax+by+c 的梯度是 (a,b)
        # # 本来要对 U_grad_new[:3,ii,:,0] 一个赋值，现在使用广播机制
        # U_grad_new[:3,:,:,0] = (coef1[:2,:]).transpose()[:, np.newaxis, :]
        # # 对edge基函数，梯度更复杂些，每个基函数都是 nquad x 2 的矩阵
        # U_grad_new[3,:,:,0] = -1/2*(coef1[:2,[0,2]]@p_new[[2,0],:]).transpose()
        # U_grad_new[4,:,:,0] = -1/2*(coef1[:2,[1,2]]@p_new[[2,1],:]).transpose()
        # U_grad_new[5,:,:,0] = -1/2*(coef1[:2,[0,1]]@p_new[[1,0],:]).transpose()
        # # 接下来是y分量的基函数取值，是前面x分量的copy到y上
        # U_grad_new[6:12,:,:,1] = U_grad_new[:6,:,:,0]
        # # 接下来是外部的基函数取值，是采用的是p_new的后3行
        # U_grad_new[12:15,:,:,0] = (coef2[:2,:]).transpose()[:, np.newaxis, :]
        # U_grad_new[15,:,:,0] = -1/2*(coef2[:2,[0,2]]@p_new[[5,3],:]).transpose()
        # U_grad_new[16,:,:,0] = -1/2*(coef2[:2,[1,2]]@p_new[[5,4],:]).transpose()
        # U_grad_new[17,:,:,0] = -1/2*(coef2[:2,[0,1]]@p_new[[4,3],:]).transpose()
        # U_grad_new[18:,:,:,1] = U_grad_new[12:18,:,:,0]
        # print("Check U_grad_procy is {}".format(np.allclose(U_grad_new,np.array(U_Proxy_grad_Val))))
        
        U_Proxy_Vals.append(U_Proxy_Val)
        p_Proxy_Vals.append(p_Proxy_Val)
        U_Proxy_grad_Vals.append(U_Proxy_grad_Val)
    return U_Proxy_Vals, p_Proxy_Vals, U_Proxy_grad_Vals, G_X_list, G_w_list

In [10]:
pwd

'/home/jovyan/work/ngs-esfem/assemble'

In [13]:
import sys
sys.path.append("/home/jovyan/work/")

In [17]:
sys.path.append("/home/jovyan/work/MDNS")

In [37]:
import numpy as np
from GenerateGeo import *
from ngsolve import *
from My_Assemble import *
from es_utils import pos_transformer
import copy
from functools import partial
import multiprocessing
from ProblemSetting import *
from contextlib import contextmanager
from pathlib import Path

def convert_to_ndarray(mat):
    return {key: np.array(value) for key, value in mat.items()}

MEASURE_TIME = True
# 定义一个上下文管理器来测量时间
@contextmanager
def time_block(label):
    if MEASURE_TIME:
        start = time.time()
    try:
        yield
    finally:
        if MEASURE_TIME:
            end = time.time()
            print(f"{label} took {end - start:.4f} seconds")

In [19]:
printnotes = True
pvdsave = True
mu0 = 1
rho = 1
Ntset = 200
mybub_opt = False
rAI = 1
T = 3 # 角速度为pi，T=3可以转到3pi角度

h = 0.2
hInterface = h
mymesh, mesh_in_circ, mesh_out_circ, Info_BND, BNDEl, Ind_V_Ext, \
    Int_ind = MeshLSQuadRect(hout=h,hin=h,hI=h,rAI=rAI,ll=(-1.5,-1.5),ur=(1.5,1.5))
N_AI = Info_BND["N_AI"]
theta0 = np.pi*2/N_AI

here


In [21]:
nquad = 4
k = 0
porder = 1
Uorder = 1 if mybub_opt else 2
ome = np.pi # moving velocity of interior domain
myt = Parameter(0)
# the exact solution satisfies the Dirichlet Boundary Condition in the interior interface (circle)
# At t = 0, initial velocity field is nonzero (but zero at interior boundary)
mu = mu0/rho
# mu* Laplace u = 2*mu*div(Du)
exactux, exactuy = exact_u(ome, myt)
exactp = exact_pfix(ome,myt)
## 之前写方程的时候漏写了2mu的系数2
RhsFunc1 = exactux.Diff(myt).Compile(realcompile=True)
RhsFunc11 = (exactux*exactux.Diff(x) + exactuy*exactux.Diff(y)) - mu*(exactux.Diff(x).Diff(x)+exactux.Diff(y).Diff(y)) \
        + 1/rho*exactp.Diff(x)

RhsFunc2 = exactuy.Diff(myt).Compile(realcompile=True)
RhsFunc21 = (exactux*exactuy.Diff(x) + exactuy*exactuy.Diff(y)) - mu*(exactuy.Diff(y).Diff(y)+exactuy.Diff(x).Diff(x)) \
        + 1/rho*exactp.Diff(y)
RhsFuncA = CF((RhsFunc1,RhsFunc2)).Compile(realcompile=True)
RhsFuncA2 = CF((RhsFunc11,RhsFunc21)).Compile(realcompile=True)

### 修改的右端项

In [22]:
myt = Parameter(0)
rhs_full = exact_rhs(ome,myt,mu0,rho)

In [23]:
tau = Parameter(0)
# FEM space for U
Vfes_int = VectorH1(mymesh, definedon="mat_int", order=Uorder, dirichlet="quat")
Vfes_ext = VectorH1(mymesh, definedon="mat_ext", order=Uorder, dirichlet="box")
Vfes_int = Compress(Vfes_int)
Vfes_ext = Compress(Vfes_ext)
Vfes_all = Vfes_int * Vfes_ext
Cspace = NumberSpace(mymesh)

V_Disp = Compress(VectorH1(mymesh, definedon="mat_int", order=1))

# FEM space for p
fes11 = Compress(H1(mymesh, definedon="mat_int", order=porder))
fes21 = Compress(H1(mymesh, definedon="mat_ext", order=porder))
nvers = [fes11.ndof,fes21.ndof] # number of vertices of interior and exterior domain (double the interface)
fes_all = fes11 * fes21

el_list_in, _, _ = BNDDofs(fes11,"Arti",fesdim=porder)
el_list_out, _, _ = BNDDofs(fes21,"Arti",fesdim=porder)

Vel_list_in, _, _ = BNDDofsBubble(Vfes_int,"Arti",fesdim=2,order=Uorder,Bub=mybub_opt)
Vel_list_out, _, _ = BNDDofsBubble(Vfes_ext,"Arti",fesdim=2,order=Uorder,Bub=mybub_opt)

# after moving N anticlockwise, the exterior becomes jj + N
NI = len(el_list_in)
# shift dofs: it is important that it performs only once!
for el in Vel_list_out:
    el.DofObj.shift_dofs(Vfes_int.ndof)

for el in el_list_out:
    el.DofObj.shift_dofs(fes11.ndof)

# %% Visual u and p
Visual_V_Disp = Compress(VectorH1(mesh_in_circ, order=1))
Visual_Int = H1(mesh_in_circ, order=Uorder)
Visual_Out = H1(mesh_out_circ, order=Uorder)
Visual_Int_1 = H1(mesh_in_circ, order=1)
Visual_Out_1 = H1(mesh_out_circ, order=1)
func_int = GridFunction(Visual_Int_1)
func_out = GridFunction(Visual_Out_1)
Visual_Int_d = VectorH1(mesh_in_circ, order=1)
Visual_Out_d = VectorH1(mesh_out_circ, order=1)
func_int_d = GridFunction(Visual_Int_d)
func_out_d = GridFunction(Visual_Out_d)
Visual_Int = Compress(Visual_Int)
Visual_Out = Compress(Visual_Out)

V_gfu_ex_x = GridFunction(Visual_Out)
V_gfu_ex_y = GridFunction(Visual_Out)
V_gfu_in_x = GridFunction(Visual_Int)
V_gfu_in_y = GridFunction(Visual_Int)
Disp_Visual = GridFunction(Visual_V_Disp)

Visual_Int_1 = Compress(H1(mesh_in_circ, order=porder))
Visual_Out_1 = Compress(H1(mesh_out_circ, order=porder))
p_gfu_ex = GridFunction(Visual_Out_1)
p_gfu_in = GridFunction(Visual_Int_1)

exact_in_x = GridFunction(Visual_Int)
exact_out_x = GridFunction(Visual_Out)
exact_in_y = GridFunction(Visual_Int)
exact_out_y = GridFunction(Visual_Out)

err_in_x = GridFunction(Visual_Int)
err_out_x = GridFunction(Visual_Out)
err_in_y = GridFunction(Visual_Int)
err_out_y = GridFunction(Visual_Out)

exact_in_p = GridFunction(Visual_Int_1)
exact_out_p = GridFunction(Visual_Out_1)
err_in_p = GridFunction(Visual_Int_1)
err_out_p = GridFunction(Visual_Out_1)

# %% 
U_n = GridFunction(Vfes_all)
p_n_all = GridFunction(fes_all)
disp_old = GridFunction(V_Disp)
disp_new = GridFunction(V_Disp)
Ini_P = GridFunction(V_Disp)
Ini_P.Set(CF((x,y)),definedon="mat_int")
Ini_Coords = pos_transformer(Ini_P)

Coords = np.array([v.point for v in mymesh.vertices])
CoordsInt = copy.deepcopy(Coords)
CoordsExt = copy.deepcopy(Coords)
Ori_CoordsInt = copy.deepcopy(CoordsInt[Int_ind,:])

In [25]:
w = GridFunction(V_Disp)

u,v = Vfes_all.TnT()
p,q = fes_all.TnT()
lam, mu2 = Cspace.TnT()
B_Cq = BilinearForm(trialspace=Cspace, testspace=fes_all)
B_Cq += lam*q[0]*dx(definedon=mymesh.Materials('mat_int')) \
    + lam*q[1]*dx(definedon=mymesh.Materials('mat_ext'))

B_UV = BilinearForm(Vfes_all,check_unused=False)
B_UV += 1/(2*tau)*(InnerProduct((u[0]),(v[0]))) * dx(definedon=mymesh.Materials('mat_int'))

# 2*mu*(Du,Dv)
B_UV_2 = BilinearForm(Vfes_all)
B_UV_2 += 1/(2*tau)*(InnerProduct((u[0]),(v[0]))) * dx(definedon=mymesh.Materials('mat_int'))
B_UV_2 += 1/2*mu*(InnerProduct(grad(u[0])+grad(u[0]).trans,grad(v[0])+grad(v[0]).trans) +
                InnerProduct(grad(u[1])+grad(u[1]).trans,grad(v[1])+grad(v[1]).trans)) * dx
# b(u;u,v)
B_UV_2 += 1/2*(InnerProduct(grad(u[0])*(U_n.components[0]-w),v[0])
            -InnerProduct(grad(v[0])*(U_n.components[0]-w),u[0])
            +InnerProduct(grad(u[1])*U_n.components[1],v[1])
            -InnerProduct(grad(v[1])*U_n.components[1],u[1]))*dx
B_UV_2 += 1/tau*(InnerProduct(u[1],v[1])) * dx(definedon=mymesh.Materials('mat_ext'))

# Assembled on the undeformed mesh
Rhs_u_old = LinearForm(Vfes_all)
Rhs_u_old += 1/tau*(InnerProduct(U_n.components[0],v[0])+
                InnerProduct(U_n.components[1],v[1])) * dx
# Source term
Rhs_F = LinearForm(Vfes_all)
Rhs_F += InnerProduct(RhsFuncA,v[0]) * dx
Rhs_F += InnerProduct(RhsFuncA,v[1]) * dx
Rhs_F += InnerProduct(RhsFuncA2,v[0]) * dx
Rhs_F += InnerProduct(RhsFuncA2,v[1]) * dx


B_Uq = BilinearForm(trialspace=Vfes_all, testspace=fes_all)
B_Vp = BilinearForm(trialspace=fes_all, testspace=Vfes_all)
B_Uq += (div(u[0])*q[0])*dx(definedon=mymesh.Materials('mat_int'))\
    + (div(u[1])*q[1])*dx(definedon=mymesh.Materials('mat_ext'))
B_Vp += -(div(v[0])*p[0])*dx(definedon=mymesh.Materials('mat_int'))\
    - (div(v[1])*p[1])*dx(definedon=mymesh.Materials('mat_ext'))

gfu = GridFunction(Vfes_all, name="u")
gfp = GridFunction(fes_all, name="p")

t_old = 0 
dt = T/Ntset    
tau.Set(dt)

def get_base_dir():
    try:
        return Path(__file__).resolve().parent
    except NameError:
        return Path.cwd()   # Jupyter: 当前 notebook 所在目录
    
# ---------------------------
# experiment name
# ---------------------------
fName = (
    f"Conv/Spatial/Ntset_{Ntset}_T_{T}_mu_{mu0}_rho_{rho}_k_{k}_rAI_{rAI}"
    .replace('.', '_')
)

# ---------------------------
# base directory (cross-platform)
# ---------------------------
BASE_DIR = get_base_dir()   # 当前脚本所在目录
Base_Dir = BASE_DIR / fName
Sub_Dir = Base_Dir / f"h_{h}"

# ---------------------------
# create directories
# ---------------------------
created = not Sub_Dir.exists()
Sub_Dir.mkdir(parents=True, exist_ok=True)
if created:
    print(f"[INFO] Created directory: {Sub_Dir.resolve()}")

# ---------------------------
# vtk output
# ---------------------------
if pvdsave:
    savefunc_obj1 = Vtk_out([T], [20], Sub_Dir / "fig1")
    savefunc_obj2 = Vtk_out([T], [20], Sub_Dir / "fig2")

myPrinter = Printer(20)
Dis_Eng, T_set = [],[]

mymesh.UnsetDeformation()
U_n.components[0].Set(CF((exactux,exactuy)),definedon=mymesh.Materials('mat_int'))
U_n.components[1].Set(CF((exactux,exactuy)),definedon=mymesh.Materials('mat_ext'))
omega, theta_old = 0, 0

[INFO] Created directory: /home/jovyan/work/ngs-esfem/assemble/Conv/Spatial/Ntset_200_T_3_mu_1_rho_1_k_0_rAI_1/h_0.2
  vtk files are saved in /home/jovyan/work/ngs-esfem/assemble/Conv/Spatial/Ntset_200_T_3_mu_1_rho_1_k_0_rAI_1/h_0.2/fig1  
  vtk files are saved in /home/jovyan/work/ngs-esfem/assemble/Conv/Spatial/Ntset_200_T_3_mu_1_rho_1_k_0_rAI_1/h_0.2/fig2  


In [28]:
t_old = 0
t_old += dt # to solve solution at this time
myt.Set(t_old) # for exact solution, Dirichlet BC, source term f
omega = ome  # constant angular velocity
theta_new = theta_old + omega*dt # analytic expression theta = ome*t**2/2
myPrinter.PrintDuring(t_old,T,T_begin=0) 

2026-02-09 11:14:33 +0800 Asia/Shanghai-Finished 0.0 per cent


* tn的旋转角度是theta_old
* 利用初始的坐标Ini_Coords和旋转的角度来计算tn的deformation向量old_coords_def 和 disp_old
* tn+1的旋转角度是theta_new
* 计算tn+1的deformation向量new_coords_def 和 disp_new
* 计算tn+1的position new_pos
* 计算tn+1的每个点的角速度向量，来构造ALE速度场w
* 获取内部区域形变后的坐标CoordsInt。

In [38]:
with time_block("ALE setting"):
    old_coords_def = Ini_Coords@np.array([[np.cos(theta_old)-1, np.sin(theta_old)],
                                        [-np.sin(theta_old), np.cos(theta_old)-1]])
    disp_old.vec.data = BaseVector(old_coords_def.flatten('F'))

    new_coords_def = Ini_Coords@np.array([[np.cos(theta_new)-1, np.sin(theta_new)],
                                        [-np.sin(theta_new), np.cos(theta_new)-1]])
    disp_new.vec.data = BaseVector(new_coords_def.flatten('F'))
    Disp_Visual.vec.data = BaseVector(new_coords_def.flatten('F'))

    new_pos = new_coords_def + Ini_Coords

    ang_vel = new_pos@np.array([[0,1],[-1,0]])*omega
    w.vec.data = BaseVector(ang_vel.flatten('F'))

    # Get the transformed interior coords, still keeping ext nodes to make the index of int nodes correct
    CoordsInt[Int_ind,:] = Ori_CoordsInt@np.array([ [np.cos(theta_new), np.sin(theta_new)],
                                                    [-np.sin(theta_new), np.cos(theta_new)]])

    mymesh.SetDeformation(disp_old)

ALE setting took 0.0042 seconds


* 时间质量项组装，并保存矩阵
* tn时刻的右端项组装
* 网格形变（得到tn+1的网格）
* 在tn+1的网格上组装 - 新的质量矩阵，源项（右端项f），粘性项，速度压力耦合项 vp 以及 压力constraint项压力积分为0 pressure_mean

#### 矩阵组装占据不少的时间

In [39]:
with time_block("Bilinear Form Assemble"):
    B_UV.Assemble()
    Rhs_u_old.Assemble()
    MatUV_old = B_UV.mat.CreateMatrix()

    mymesh.SetDeformation(disp_new)
    mesh_in_circ.SetDeformation(Disp_Visual)
    for ii, bform in enumerate([B_UV,Rhs_F,B_UV_2,B_Vp,B_Cq]):
        # Assemble the matrix 占据了一些时间。可以优化的部分包括外部的fix的domain可以不用更新
        bform.Assemble()

    # Set Dirichlet Boundary Conditions
    gfu.components[0].Set(CF((exactux,exactuy)),definedon=mymesh.Boundaries('quat'))
    gfu.components[1].Set(CF((exactux,exactuy)),definedon=mymesh.Boundaries('box'))

Bilinear Form Assemble took 0.0493 seconds


* N代表了当前的旋转角度对应的单元错位的次数
* 边界对应的内层上有NI个单元：每个单元对应两个外部单元；没有旋转的时候就是 list_out 的 jj 和 jj+1；有旋转错位之后再加N
* item 收录 第jj个pair对应的内外单元

In [40]:
with time_block("Items Collect: "):
    N = int(np.floor(theta_new/theta0))
    items = []
    for jj in range(NI): # 边界上有NI个单元
        int_el = el_list_in[jj]
        ext_el_0 = el_list_out[np.mod(jj+N,NI)]
        ext_el_1 = el_list_out[np.mod(jj+N+1,NI)]

        Vint_el = Vel_list_in[jj]
        Vext_el_0 = Vel_list_out[np.mod(jj+N,NI)]
        Vext_el_1 = Vel_list_out[np.mod(jj+N+1,NI)]
        items.append((int_el, ext_el_0, ext_el_1, Vint_el, Vext_el_0, Vext_el_1, jj))

Items Collect:  took 0.0005 seconds


\begin{equation}
A_h^{n+1}\left(\mathbf{u}_h^n \circ\left(\phi_{h+}^n\right)^{-1} ; \mathbf{u}_h^{n+1}, \mathbf{v}_h\right)
\end{equation}

\begin{align*}
A(\mathbf{z} ; \mathbf{u}, \mathbf{v}) :=\;
& \sum_{i=\pm}
\left( 2\mu\, \mathbb{D}\mathbf{u}_i , \mathbb{D}\mathbf{v}_i \right)_{\Omega_i}  \\
& + \frac{1}{2}
\left\langle
\{\!\{ (\mathbf{z}\cdot\mathbf{n}) \mathbf{u} \}\!\},
[\![ \mathbf{v} ]\!]
\right\rangle_{\Gamma}
- \frac{1}{2}
\left\langle
\{\!\{ (\mathbf{z}\cdot\mathbf{n}) \mathbf{v} \}\!\},
[\![ \mathbf{u} ]\!]
\right\rangle_{\Gamma} \\
& - 2\mu
\left\langle
\{\!\{ \mathbb{D}(\mathbf{u}) \mathbf{n} \}\!\},
[\![ \mathbf{v} ]\!]
\right\rangle_{\Gamma}
+ 2\mu
\left\langle
\{\!\{ \mathbb{D}(\mathbf{v}) \mathbf{n} \}\!\},
[\![ \mathbf{u} ]\!]
\right\rangle_{\Gamma}.
\end{align*}


* 用tn时刻的un来计算线性化的对流项

### 是否可以改进成先写矩阵，然后直接将unvec乘进去的形式？
* argu0只收集了第一个pair（参考pair）的信息（单元信息，un的向量信息，以及必要的参数，最后是pair的编号）

In [41]:
with time_block("Basis Assemble Old: "):
    unvec = U_n.vec.FV().NumPy()
    argu0 = tuple(list(items[0][0:6])+[unvec, theta0, k, mu, nquad, rho, mybub_opt]+[items[0][-1]])
    # Parallel execution of process_item using the Pool
    # Grad 是nabla 5个元素输出
    ProxyVals0 = GetBasisInfo(*argu0,theta_new,CoordsInt, CoordsExt)

Basis Assemble Old:  took 0.0252 seconds


In [42]:
with time_block("Basis Assemble New: "):
    pIntCoord = CoordsInt[argu0[0].vnr,:]
    pExtCoord0 = CoordsExt[argu0[1].vnr,:]
    pExtCoord1 = CoordsExt[argu0[2].vnr,:]
    Info_X, Info_W = QuadInfo(theta_new,theta0,nquad,1)
    # ps.shape = (2,6,4)
    # Us.shape = (2,24,4,2)
    # U_grads.shape = (2,24,4,2,2)
    ps, Us, U_grads = GetVals(pIntCoord, pExtCoord0, pExtCoord1, Info_X)
    ProxyVals0 = [Us, ps, U_grads, Info_X, Info_W]

Basis Assemble New:  took 0.0046 seconds


In [34]:
UdofsList_all = []
pdofsList_all = []
for jj in range(NI): # 边界上有NI个单元
    int_el = el_list_in[jj]
    ext_el_0 = el_list_out[np.mod(jj+N,NI)]
    ext_el_1 = el_list_out[np.mod(jj+N+1,NI)]

    Vint_el = Vel_list_in[jj]
    Vext_el_0 = Vel_list_out[np.mod(jj+N,NI)]
    Vext_el_1 = Vel_list_out[np.mod(jj+N+1,NI)]

    # 整体处理的需要，列出所有的dofs
    UDof_els:List[femDofel] = [Vint_el, Vext_el_0, Vext_el_1]
    UdofsList = np.array([el.DofObj.GetDofs() for el in UDof_els]).flatten() 
    UdofsList_all.append(UdofsList)
    pDof_els:List[femDofel] = [int_el, ext_el_0, ext_el_1]
    pdofsList = np.array([el.DofObj.GetDofs() for el in pDof_els]).flatten() 
    pdofsList_all.append(pdofsList)

# 遍历界面内边界单元，给出unvec的local切片；shape=UdofsList
# 每行代表一个element组，collect了这个element组里面的速度系数
result = unvec[UdofsList_all]
# 方法四：不用循环
# GenerateUPMat_Faster(*arguments[0])
with time_block("General Tensor Product: "):
    Up_Mat_all, UV_Mat_all = TimeIndependent(*ProxyVals0, result, theta0, NI, nquad=4, rho=1, mu=1)

In [35]:
with time_block("Domain Matrix2: "):

    MatRow_new, MatCol_new, MatVal_new = [], [], []


    Sp_Row_List, Sp_Col_List, Sp_Val_List = [], [], []
    Up_Row_List, Up_Col_List, Up_Val_List = [], [], []
    qp_Row_List, qp_Col_List, qp_Val_List = [], [], []

    for ii in range(NI):
        row_idx, col_idx, values = GetCooInfo(UV_Mat_all[ii],UdofsList_all[ii],UdofsList_all[ii])
        Sp_Row_List.extend([row_idx,col_idx])
        Sp_Col_List.extend([col_idx,row_idx])
        Sp_Val_List.extend([values, -values])

        row_idx, col_idx, values = GetCooInfo(Up_Mat_all[ii],UdofsList_all[ii],pdofsList_all[ii])
        Up_Row_List.append(row_idx)
        Up_Col_List.append(col_idx)
        Up_Val_List.append(values) 

    MatRow_new.append(np.concatenate(Sp_Row_List))
    MatRow_new.append(np.concatenate(Up_Row_List))
    MatRow_new.append(np.concatenate(Up_Col_List)+Vfes_all.ndof)

    MatCol_new.append(np.concatenate(Sp_Col_List))
    MatCol_new.append(np.concatenate(Up_Col_List)+Vfes_all.ndof)
    MatCol_new.append(np.concatenate(Up_Row_List))

    MatVal_new.append(np.concatenate(Sp_Val_List))
    MatVal_new.append(np.concatenate(Up_Val_List)/rho)
    MatVal_new.append(-1*np.concatenate(Up_Val_List))

    for mymat in [MatUV_old,B_UV_2.mat]:
        row, col, val = map(np.array, mymat.COO())
        MatRow_new.append(row)
        MatCol_new.append(col)
        MatVal_new.append(val)

    for mymat in [B_Vp.mat]:
        row, col, val = map(np.array, mymat.COO())
        MatRow_new.append(row)
        MatCol_new.append(col+Vfes_all.ndof)
        MatVal_new.append(val/rho)

        MatRow_new.append(col+Vfes_all.ndof)
        MatCol_new.append(row)
        MatVal_new.append(-val)

    for mymat in [B_Cq.mat]:
        row, col, val = map(np.array, mymat.COO())
        MatRow_new.append(row+Vfes_all.ndof)
        MatCol_new.append(col+Vfes_all.ndof+fes_all.ndof)
        MatVal_new.append(val)

        MatRow_new.append(col+Vfes_all.ndof+fes_all.ndof)
        MatCol_new.append(row+Vfes_all.ndof)
        MatVal_new.append(-val)
    MatRow2 = np.concatenate(MatRow_new)
    MatCol2 = np.concatenate(MatCol_new)
    MatVal2 = np.concatenate(MatVal_new)

In [36]:
rcn = Vfes_all.ndof + fes_all.ndof + 1

if t_old == dt:
    print("dims of the final matrix is {} 非零元素的个数是 {}".format(rcn,len(MatVal2)))

with time_block("Matrix Assemble: "):
        #%% Assemble Whole matrix
    K = NGMO.myCOO(MatRow2, MatCol2, MatVal2, rcn, rcn, "ngsolve")
    solnp = np.hstack([gfu.vec.FV().NumPy(), gfp.vec.FV().NumPy(), np.array([0.0])])
    sol = BaseVector(solnp)
    rhsnp = np.hstack([Rhs_u_old.vec.FV().NumPy()+Rhs_F.vec.FV().NumPy(),
            np.array([0.0]*(fes_all.ndof+1))]) # 分别代表p方程和p积分为0的multiplier的右端项
    rhsval = BaseVector(rhsnp)
    FreeDofs = BitArray(list(Vfes_all.FreeDofs())+list(fes_all.FreeDofs())+list([True]))
with time_block("Matrix Solving: "):
    resDiri = BaseVector(K.Inverse(FreeDofs,inverse='umfpack')*(rhsval - K*sol)+sol)

dims of the final matrix is 2760 非零元素的个数是 198722


In [ ]:
U_n.vec.data = BaseVector(resDiri.FV().NumPy()[:Vfes_all.ndof])
DistributeSol(resDiri,[V_gfu_in_x,V_gfu_in_y,V_gfu_ex_x,V_gfu_ex_y,
                    p_gfu_in, p_gfu_ex])